In [33]:
import os
import torch
import warnings

from torch import nn # 신경망을 구축하기 위한 모듈
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import lightning as L

warnings.filterwarnings("ignore")

# Define the PyTorch nn.Modules

In [35]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28*28, 64), nn.ReLU(), nn.Linear(64,3))
        
    def forward(self, x):
        return self.l1(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3,64), nn.ReLU(), nn.Linear(64, 28*28))
        
    def forward(self, x):
        return self.l1(x)

# Define a LightningModule

In [36]:
class LitAutoEncoder(L.LightningModule):
    '''
    오토인코더 
    - 정의 : 입력을 encoder를 통해 정보를 압축하고, decoder를 통해 입력과 유사한 형태로 재구성 
    - 목적 : 입력 데이터의 효과적인 표현을 학습하는데 목적이 있다.
    '''
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def training_step(self, batch, batch_idx):
        # 각 학습 batch에 대해 호출되며, 해당 배치를 처리하고 손실을 계산하는 등의 학습과 관련된 작업 수행
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss
    
    def configure_optimizers(self):
        '''
        optimizer : 가중치를 효율적으로 업데이트하는 최적화 알고리즘 
        adam : 모멘텀과 이동평균을 고려한 최적화 알고리즘 => 빠른 수렴 속도 
        - 모멘텀 : (관성) 가중치를 한 방향으로 계속해서 움직이게 함
        - 이동평균 : 최근 데이터에 높은 가중치를 둔다.
        '''
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Define the trainig dataset

In [37]:
# 데이터 셋
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
# 데이터로더
train_loader = DataLoader(dataset, batch_size=10)

# Train the model

In [ ]:
# model
autoencoder = LitAutoEncoder(Encoder(), Decoder())

# train model
trainer = L.Trainer(fast_dev_run=10)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)